In [ ]:
import mcstasscript as ms

### Calculator is what I call an instrument object

In [ ]:
calculator = ms.McStas_instr(name="demo_instrument")

### Can define parameters (libpyvinyl parameter object and collection)

In [ ]:
source_energy = calculator.add_parameter("double", "source_energy", unit="meV", comment="Source mean energy")
source_energy.add_interval(3.1, None, intervals_are_legal=True)

source_height = calculator.add_parameter("double", "source_height", unit="cm", comment="Height of source")
source_height.add_interval(0.01, 0.2, intervals_are_legal=True)

sample_height = calculator.add_parameter("sample_height", unit="cm", comment="Height of sample")
sample_height.add_interval(0.0, 0.05, intervals_are_legal=True)

source_energy_spread = calculator.add_declare_var("double", "source_energy_spread")
calculator.append_initialize("source_energy_spread = 0.02*source_energy;")

In [ ]:
calculator.show_parameters()

In [ ]:
print(calculator.parameters)

### Can add components to the simulation as usual

In [ ]:
src = calculator.add_component("source", "Source_div")

In [ ]:
src.xwidth = 0.12
src.yheight = source_height
src.E0 = source_energy
src.dE = source_energy_spread
src.focus_aw = 3.0
src.focus_ah = 4.0

In [ ]:
sample = calculator.add_component("powder", "PowderN")

In [ ]:
sample.set_AT(1, RELATIVE=src)
sample.reflections='"Ni.laz"'
sample.radius = 0.01
sample.yheight = sample_height

In [ ]:
calculator.available_components("monitors")

In [ ]:
calculator.component_help("Cyl_monitor")

In [ ]:
cyl = calculator.add_component("cyl_monitor", "Cyl_monitor")

In [ ]:
cyl.nr = 200
cyl.filename = '"cylinder.dat"'
cyl.yheight = 0.2
cyl.radius = 0.5
cyl.restore_neutron = 1
cyl.set_AT(0, RELATIVE=sample)

In [ ]:
mon = calculator.add_component("acceptance_horizontal", "DivPos_monitor")

In [ ]:
mon.xwidth = 0.2
mon.yheight = 0.2
mon.maxdiv = 30.0
mon.filename = '"acceptance_h.dat"'
mon.restore_neutron = 1
mon.nb = 300
mon.ndiv = 300
mon.set_AT(0.1, RELATIVE=sample)

In [ ]:
calculator.show_components()

### Old syntax

In [ ]:
old_data = calculator.run_full_instrument(parameters={"source_energy": 320, "source_height":0.025, "sample_height":0.04},
                               ncount=2E6, foldername="run_full_instrument_path")

### New libpyvinyl syntax

In [ ]:
calculator.set_parameters(source_energy=320, source_height=0.025, sample_height=0.04)

In [ ]:
calculator.settings(ncount=5E6, mpi=2, output_path="path_to_output")

### Run with backengine

In [ ]:
data = calculator.backengine()

### Get data later

In [ ]:
calculator.output.get_data()["data"]

### Plot data

In [ ]:
ms.make_sub_plot(data, log=True, cut_max=0.011, orders_of_mag=1.5)

### Store in dump file

In [ ]:
calculator.dump("dump_file.dmp")

### Load from dump file

In [ ]:
from_dump = ms.McStas_instr.from_dump('dump_file.dmp')

In [ ]:
print(from_dump.name)
from_dump.show_parameters()
from_dump.show_components()
from_dump.write_full_instrument()

In [ ]:
with open("demo_instrument.instr", "r") as f:
    instrument_file = f.read()
print(instrument_file)